In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import matplotlib.pyplot as plt

from eo_data_tools.stac_configuration import de_australia_stac_config
from eo_data_tools.spatial import xr_rasterize, xr_vectorize
from eo_data_tools.raster_base import RasterBase, QueryParams, LoadParams

In [ ]:
de_australia_stac_config.list_collections()

In [ ]:
query_params = QueryParams(
    bbox=(142.1, -32.6, 142.80, -32.1),
    start_date="2000",
    end_date="2000",
)

load_params = LoadParams(
    crs="EPSG:3577",
    resolution=10,
    bands=("frequency"),
)

stac_raster = RasterBase.from_stac_query(
    config=de_australia_stac_config,
    collections=["ga_ls_wo_fq_cyear_3"],
    query_params=query_params,
    load_params=load_params,
)

In [ ]:
# Select pixels that are classified as water > 25 % of the time
water_bodies = stac_raster.data.frequency > 0.25

# Plot the data
water_bodies.plot(size=5)

In [ ]:
gdf = xr_vectorize(da=water_bodies, mask=water_bodies.data == 1)

gdf.plot(figsize=(6, 6))

In [ ]:
water_bodies_again = xr_rasterize(gdf=gdf, da=water_bodies)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
water_bodies.plot(ax=axes[0])
water_bodies_again.plot(ax=axes[1])
axes[0].set_title("Original waterbodies")
axes[1].set_title("Rasterized waterbodies")